In [47]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import joblib
tqdm.pandas()

# LOAD DATA

In [48]:
官方指認欄位 = ['2','6','10','12','13','15','18','19','21','22','25','26','36','37','39','48']
nbrs = joblib.load('../model/nbrs.pkl')
X_pca = joblib.load('../model/X_pca_for_knn.pkl')
df_groupby_chid_preprocessed = pd.read_feather('../data/df_groupby_chid_preprocessed.feather')
df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
df = df.loc[df.dt >= 12] # 取最近12個月資料
test_data = pd.read_feather('../data/需預測的顧客名單及提交檔案範例.feather')

display(df.head())
display(test_data.head())

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
13188928,12.0,10115966.0,15,1.0,9678.450445,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188938,12.0,10115966.0,18,1.0,9624.614033,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188944,12.0,10115966.0,25,1.0,2338.181448,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188946,12.0,10115966.0,26,2.0,16627.150110,2.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188957,12.0,10115966.0,37,13.0,18485.113446,3.0,10.0,0.0,0.0,0.27,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


In [53]:
def chid2answer(chid): #chid到answer的映射
    a = df.loc[df['chid']==chid,'shop_tag'].value_counts().to_frame()
    a['在指認欄位'] = False
    if len(list(set(a.index)&set(官方指認欄位))) != 0: #如果跟官方指定欄位有交集
        a.loc[list(set(a.index)&set(官方指認欄位)),'在指認欄位'] = True #有交集的部份做記號
    answer = a[a['在指認欄位']==True].index.tolist()[:3] # 取有交集的部份前三名返回
    return answer

def predict_function(chid): # 預測函數
    answer = chid2answer(chid) # 根據這個chid做預測
    if len(answer) == 3: # 如果成功找到三個直接return
        return answer
    else:
        remain = 3-len(answer) # 否則計算還缺多少
        idx = df_groupby_chid_preprocessed.loc[df_groupby_chid_preprocessed.chid==chid].index[0] # 根據chid找到該筆樣本的"idx"
        distances, indices = nbrs.kneighbors(X_pca[[idx]]) # 根據該樣本的idx找到該筆樣本的"PCA特徵"然後取得"鄰居的indices"
        chid_list = df_groupby_chid_preprocessed.loc[indices[0][-(nbrs.n_neighbors-1):]]['chid'].values.tolist() # 根據"鄰居的indices"取得"chid_list(鄰居們)"
        answer_list = [chid2answer(chid) for chid in chid_list]# 根據"chid_list"取得"answer_list"
        # 將answer_list做一維展開========
        answer_list_ = []
        for i in answer_list:
            answer_list_.extend(i)
        answer_list = answer_list_
        # 將answer_list做一維展開========
        answer_list = list(set(answer_list) - set(answer)) # 如果"answer"裡面已經有了則"去除"
        for _ in range(remain):
            if len(answer_list) != 0: #如果answer_list不等於0
                shop_tag = answer_list[0] # 從answer_list選第一個shop_tag(選第一個是因為distances比較近)
                answer.append(shop_tag) # 加入shop_tag至answer
                answer_list.remove(shop_tag) # 用過的shop_tag記得移除
            else:
                shop_tag = np.random.choice(官方指認欄位)
                answer.append(shop_tag) # 加入shop_tag至answer
        return answer

In [60]:
submit = test_data.copy().head(42)
display(submit)
answer_list = submit['chid'].progress_apply(predict_function)
answer_list = answer_list.to_frame()['chid']
for i in [0,1,2]:
    submit[f'top{i+1}'] = answer_list.apply(lambda x:x[i]).values
display(submit)

,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26
5,10439550,22,26,18
6,10285755,12,13,36
7,10335334,26,18,48
8,10072243,15,13,6
9,10274006,22,6,39


100%|██████████| 42/42 [00:07<00:00,  5.85it/s]


,chid,top1,top2,top3
0,10128239,37,25,26
1,10077943,36,15,13
2,10277876,15,10,36
3,10364842,37,10,2
4,10392717,48,15,2
5,10439550,2,48,18
6,10285755,39,15,19
7,10335334,12,36,19
8,10072243,21,48,18
9,10274006,12,48,37


In [61]:
# test ok